# Using pyco2stats: Sinclair Method

## Interactive Implementation of the Sinclair Method for Population Partitioning

The **Sinclair method**, originally introduced by Sinclair (1974), is a well-established statistical procedure used to partition datasets of non-Gaussian, polymodal values into two or more log-normal subpopulations. Traditionally, this partitioning is achieved through the graphical analysis of a cumulative probability plot of the data. The method proves particularly useful in fields like geochemistry for separating a background population from one or more anomalous populations that may indicate mineralization or degassing.

This Jupyter notebook presents an interactive implementation of the Sinclair method, specifically as described by Chiodini et al. (1998), utilizing the `pyco2stats` library. Recognizing that data complexity varies, this tool is designed to allow managing up to four distinct populations. For each potential population, the user can interactively define its estimated log-mean, log-standard deviation, and its fraction (or mixing proportion) of the total dataset. The main aim of this interactive approach is to visually and statistically find the combination of log-normal populations whose cumulative distribution best fits the observed distribution of the raw data.

---

### References

* Chiodini, G., Frondini, F., Cardellini, C., Granieri, F., Tranne, C. A., Ventura, G., & Caliro, S. (1998). CO2 degassing in the Alban Hills volcanic area, Central Italy. *Journal of Geophysical Research: Solid Earth*, *103*(B11), 27509-27518.
* Sinclair, A.J. (1974) Selection of thresholds in geochemical data using probability graphs. J. Geochem. Expl., 3. 129–49.


# To open this notebook in a interactive way please click [here](https://mybinder.org/v2/gh/AIVolcanoLab/pyco2stats/88edfa23d843b043cb9bbdb4c41d09b69ce342cc?urlpath=lab%2Ftree%2Fdocs%2Fnotebooks%2FSinclair_Method_rev_1.ipynb)

## Example suing Visualize_Mpl and Matplotlib

In [1]:
import pyco2stats as PyCO2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
widgets.BoundedIntText(value=1, min=0, max=5, description='N')

from IPython.display import display, clear_output

plt.rcParams['font.family'] = ['Times New Roman'] 
plt.rcParams['font.size'] = 12 

# my synthetic sample
my_means = np.array([0.7, 2])
my_stds = np.array([0.6, 0.4])
my_weights = np.array([0.4, 0.6])

my_sample = PyCO2.GMM.sample_from_gmm(n_samples= 5000, means=my_means, 
                                     stds=my_stds, weights=my_weights, random_state=43)

my_dataset = df = pd.DataFrame(my_sample, columns=['log10CO2'])

# The following section of code aims at defining the interactive experience

# defining the interactive experience start
# Maximum number of populations we want to support
max_populations = 4

# Initial parameters
initial_population_count = 3
initial_meds = [0.7, 2, 1.0, 1.5, 2.2] # Ensure initial values match step if possible for cleaner display
initial_stds = [0.60, 0.40, 0.45, 0.50, 0.80] # Ensure initial values match step if possible
initial_fds = [0.4, 0.60, 0.50, 0.40, 0.30] # Ensure initial values match step if possible

# Create interactive widgets using Textboxes
population_count_textbox = widgets.BoundedIntText(
    value=initial_population_count, min=1, max=max_populations,
    description='Populations:', disabled=False, layout=widgets.Layout(width='auto')
)



# Set the step for FloatText widgets
float_step = 0.05

# Create lists to hold the textbox widgets
meds_textboxes = []
stds_textboxes = []
fds_textboxes = []

# Create textboxes for each population up to max_populations
for i in range(max_populations):
    meds_textboxes.append(widgets.BoundedFloatText(value=initial_meds[i], min=0.1, max=10.0, step=float_step,
                                                   disabled=False, layout=widgets.Layout(width='100px')))
    stds_textboxes.append(widgets.BoundedFloatText(value=initial_stds[i], min=0.1, max=2.0, step=float_step,
                                                   disabled=False, layout=widgets.Layout(width='100px')))
    fds_textboxes.append(widgets.BoundedFloatText(value=initial_fds[i], min=0.0, max=1.0, step=float_step,
                                                  disabled=False, layout=widgets.Layout(width='100px')))

# Create an output widget to capture the plot
output = widgets.Output()


# Create HBoxes for each parameter label and textbox pair
med_widgets = [widgets.HBox([widgets.Label(f'Med {i+1}:', layout=widgets.Layout(width='50px')), meds_textboxes[i]], layout=widgets.Layout(align_items='center')) for i in range(max_populations)]
std_widgets = [widgets.HBox([widgets.Label(f'Std {i+1}:', layout=widgets.Layout(width='50px')), stds_textboxes[i]], layout=widgets.Layout(align_items='center')) for i in range(max_populations)]
fd_widgets = [widgets.HBox([widgets.Label(f'Fd {i+1}:', layout=widgets.Layout(width='50px')), fds_textboxes[i]], layout=widgets.Layout(align_items='center')) for i in range(max_populations)]


# Arrange widgets for each population block
population_blocks = []
for i in range(max_populations):
    block = widgets.VBox([
        med_widgets[i],
        std_widgets[i],
        fd_widgets[i]
    ], layout=widgets.Layout(border='1px solid #e0e0e0', padding='2px', margin='1px 0')) # Add border and padding for clarity

    population_blocks.append(block)

# Create horizontal rule widgets
horizontal_rules = [widgets.HTML(value="<hr>", layout=widgets.Layout(margin='2px 0')) for _ in range(max_populations - 1)]

# Combine the widgets and horizontal rules into a single VBox for the left panel
# Start with the population count textbox
textboxes_content = [population_count_textbox]

# Add population blocks and horizontal rules alternately
for i in range(max_populations):
    textboxes_content.append(population_blocks[i])
    if i < max_populations - 1:
        textboxes_content.append(horizontal_rules[i])

textboxes_container = widgets.VBox(textboxes_content, layout=widgets.Layout(width='33%', margin='0 2px 0 0', overflow='auto')) # Added right margin and overflow


# Function to update the visibility of parameter textboxes and horizontal rules
def update_textbox_visibility(population_count):
    # Ensure population_count is within valid range
    current_pop_count = max(1, min(population_count, max_populations))
    # We don't update the textbox value here to avoid observer loops.

    # Control visibility of population blocks
    for i in range(max_populations):
        if i < current_pop_count:
            population_blocks[i].layout.display = 'block' # Show the block
        else:
            population_blocks[i].layout.display = 'none' # Hide the block

    # Control visibility of horizontal rules
    for i in range(max_populations - 1):
        # A rule is visible if the block before it is visible
        # which means the number of populations is greater than i
        if i < current_pop_count - 1:
            horizontal_rules[i].layout.display = 'block' # Show the rule
        else:
            horizontal_rules[i].layout.display = 'none' # Hide the rule

# defining the interactive experience stop

# Function to update the plot based on the widget values
def update_plot(*args):
    with output:
        # Clear the output before plotting new graph
        clear_output(wait=True)

        # Get the current values from the textboxes
        population_count = population_count_textbox.value
        # Ensure population_count is within bounds for slicing the lists
        current_pop_count = max(1, min(population_count, max_populations))

        meds = [textbox.value for textbox in meds_textboxes[:current_pop_count]]
        stds = [textbox.value for textbox in stds_textboxes[:current_pop_count]]
        fds = [textbox.value for textbox in fds_textboxes[:current_pop_count]]

        # Normalize fds so they sum to 1
        fds_sum = sum(fds)
        if fds_sum != 0:
            fds = [fd / fds_sum for fd in fds]
        # Important: Do NOT update the fds_textboxes values here
        # based on the normalized fds list. This would create a loop
        # as changing the textbox value triggers the observer again.
        # The normalization is applied internally for the plot only.

        # Create the figure and axis
        fig, ax = plt.subplots(figsize=(12, 7)) # Adjusted figure size for layout

        # Plot raw data with improved style
        if not my_dataset.empty:
             PyCO2.Visualize_Mpl.pp_raw_data(my_dataset.log10CO2, marker='o', ax=ax, s=5, c='#FF5733', alpha=0.7, label='Raw Data')
        else:
             ax.text(0.5, 0.5, "No data available", horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)


        # Plot combined population with enhanced styling
        if current_pop_count > 0:
            # The meds, stds, fds lists are already sliced above based on current_pop_count
            current_meds = meds
            current_stds = stds
            current_fds = fds # These are already normalized within the function

            if current_meds and current_stds and current_fds: # Check if lists are not empty
                 PyCO2.Visualize_Mpl.pp_combined_population(current_meds, current_stds, current_fds, ax=ax, linestyle='-', linewidth=3, color='#3498DB', label='Combined Population')

                 # Plot single populations with enhanced styling
                 for mean, std in zip(current_meds, current_stds):
                    PyCO2.Visualize_Mpl.pp_one_population(mean, std, z_range=(-3.5, 3.5), ax=ax, linestyle='--', linewidth=1)
            else:
                 ax.text(0.5, 0.6, "Please define population parameters", horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

        ax.set_xlim(-3.5,3.5)
        # Plot percentiles
        PyCO2.Visualize_Mpl.pp_add_percentiles(ax=ax, percentiles='full', linestyle='-.', linewidth=1, color='#2ECC71', label_size=12)

        # Adding titles and labels
        ax.set_xlabel('Sigma-Values', fontsize=16)
        ax.set_ylabel('Values', fontsize=16)

        # Adding legend and grid
        ax.legend(loc='best', fontsize=14)
        ax.grid(True, linestyle='--', alpha=0.6)

        # Adjust layout to prevent labels overlapping
        plt.tight_layout()

        # Show the plot
        plt.show()


# Arrange the widgets box and the output box side by side using HBox
main_layout = widgets.HBox([textboxes_container, output], layout=widgets.Layout(width='100%', align_items='flex-start')) # Align items to top


# Connect the update functions to the widgets
population_count_textbox.observe(lambda change: update_textbox_visibility(change['new']), names='value')
population_count_textbox.observe(update_plot, names='value')

# Observe changes on all parameter textboxes
all_parameter_textboxes = meds_textboxes + stds_textboxes + fds_textboxes
for textbox in all_parameter_textboxes:
    textbox.observe(update_plot, names='value')


# Display the main layout
display(main_layout)

# Initialize textbox visibility and plot
update_textbox_visibility(initial_population_count)
update_plot() # Initial plot rendering

## Example suing Visualize_Plotly and Plotly

In [2]:
import numpy as np
import pandas as pd
import pyco2stats as PyCO2
import dash_bootstrap_components as dbc
import dash_daq as daq
from dash import Dash, dcc, html, dash_table, Input, Output, callback_context
import plotly.graph_objects as go

# ── 1) Synthetic data ───────────────────────────────────────────
my_means, my_stds = np.array([0.7, 2.0]), np.array([0.6, 0.4])
my_weights        = np.array([0.4, 0.6])
df = pd.DataFrame(
    PyCO2.GMM.sample_from_gmm(100, my_means, my_stds, my_weights, random_state=43),
    columns=['log10CO2']
)

# ── 2) Constants & init params ───────────────────────────────────
AX_MIN,AX_MAX = -3.5, 3.5
Y_MIN,Y_MAX   = df.log10CO2.min() - 0.5, df.log10CO2.max() + 0.5
X0,X1         = -3.0, 3.0
OFFSET        = 0.05
mi_x          = np.linspace(AX_MIN, AX_MAX, 600)

init_params = [
    {'name':'Pop 1','mu':0.7,'sigma':0.6,'weight':0.4,'color':'firebrick'},
    {'name':'Pop 2','mu':2.0,'sigma':0.4,'weight':0.6,'color':'navy'}
]

# ── 3) Build Dash app ─────────────────────────────────────────────
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# ── 4) Editable DataTable ─────────────────────────────────────────
table_columns = [
    {'name':'Population','id':'name','presentation':'markdown'},
    {'name':'μ','id':'mu','type':'numeric','format':{'specifier':'.2f'}},
    {'name':'σ','id':'sigma','type':'numeric','format':{'specifier':'.2f'}},
    {'name':'Weight','id':'weight','type':'numeric','format':{'specifier':'.2f'}}
]
params_table = dash_table.DataTable(
    id='params-table',
    columns=table_columns,
    data=[p.copy() for p in init_params],
    editable=True,
    style_cell={'textAlign':'center','padding':'4px'},
    style_header={'fontWeight':'bold'},
    style_table={'overflowX':'auto'}
)

# ── 5) Panel for each population ──────────────────────────────────
def pop_panel(index):
    p = init_params[index]
    return html.Div([
        html.Div(p['name'], style={'fontWeight':'bold','textAlign':'center','marginBottom':'0.5rem'}),
        html.Div('Adjust σ', style={'textAlign':'center','marginBottom':'0.25rem'}),
        daq.Knob(
            id=f'knob-sigma-{index}',
            min=0, max=2, value=p['sigma'],
            size=70, color=p['color']
        ),
        html.Div('Adjust μ', style={'textAlign':'center','marginTop':'1rem','marginBottom':'0.25rem'}),
        daq.Knob(
            id=f'knob-mean-{index}',
            min=AX_MIN, max=AX_MAX, value=p['mu'],
            size=70, color=p['color']
        ),
    ], style={'display':'flex','flexDirection':'column','alignItems':'center'})

# ── 6) Figure factory ─────────────────────────────────────────────
def make_fig(params):
    fig = go.Figure(layout=dict(
        width=800, height=500,
        margin=dict(l=40, r=40, t=20, b=40),
        xaxis={'range':[AX_MIN,AX_MAX],'fixedrange':True,'title':'σ-value (z)'},
        yaxis={'range':[Y_MIN,Y_MAX],'fixedrange':True,'title':'log₁₀(CO₂)'},
        legend={'x':0.8,'y':0.1,'bgcolor':'rgba(255,255,255,0.8)'}
    ))
    PyCO2.Visualize_Plotly.pp_raw_data(
        df.log10CO2, fig,
        marker_kwargs={'size':8,'opacity':0.6,'color':'gray'}
    )
    for i,p in enumerate(params):
        z0, z1 = X0 + i*OFFSET, X1 - i*OFFSET
        y0, y1 = p['sigma']*z0 + p['mu'], p['sigma']*z1 + p['mu']
        fig.add_shape(type='line',
                      x0=z0, y0=y0, x1=z1, y1=y1,
                      line={'color':p['color'],'width':3},
                      editable=True, name=p['name'])
    mus   = np.array([p['mu']    for p in params])
    sigs  = np.array([p['sigma'] for p in params])
    wts   = np.array([p['weight'] for p in params])
    cdf   = PyCO2.sinclair.Sinclair.combine_gaussians(mi_x, mus, sigs, wts)
    sigv  = PyCO2.sinclair.Sinclair.cumulative_to_sigma(cdf)
    fig.add_trace(go.Scatter(
        x=sigv, y=mi_x, mode='lines',
        line={'color':'green','width':3}, name='Combined'
    ))
    return fig

base_fig = make_fig(init_params)

# ── 7) App layout ─────────────────────────────────────────────────
app.layout = dbc.Container(fluid=True, style={'padding':'20px'}, children=[
    # First row: Pop1 panel (2/12), Pop2 panel (2/12), Plot (8/12)
    dbc.Row([
        dbc.Col(pop_panel(0), width=2),
        dbc.Col(pop_panel(1), width=2),
        dbc.Col(
            dcc.Graph(
                id='plot',
                figure=base_fig,
                config={'edits':{'shapePosition':True}}
            ),
            width=8
        ),
    ], align='start', justify='start'),

    # Second row: full-width table
    dbc.Row([
        dbc.Col(params_table, width=12)
    ], style={'marginTop':'2rem'})
])

# ── 8) Callback: sync knobs/table & dragging lines ──────────────
@app.callback(
    Output('plot','figure'),
    Output('params-table','data'),
    Output('knob-sigma-0','value'),
    Output('knob-sigma-1','value'),
    Output('knob-mean-0','value'),
    Output('knob-mean-1','value'),
    Input('params-table','data'),
    Input('knob-sigma-0','value'),
    Input('knob-sigma-1','value'),
    Input('knob-mean-0','value'),
    Input('knob-mean-1','value'),
    Input('plot','relayoutData'),
    prevent_initial_call=True
)
def update_plot(table_data, s0, s1, m0, m1, relayout):
    params = [dict(p) for p in table_data]
    params[0]['sigma'], params[1]['sigma'] = s0, s1
    params[0]['mu'],    params[1]['mu']    = m0, m1

    trig = callback_context.triggered[0]['prop_id']
    if trig.startswith('plot.relayoutData') and relayout:
        for i in (0,1):
            y0 = relayout.get(f'shapes[{i}].y0')
            y1 = relayout.get(f'shapes[{i}].y1')
            if y0 is not None and y1 is not None:
                z0, z1 = X0 + i*OFFSET, X1 - i*OFFSET
                sigma_new = (y1 - y0)/(z1 - z0)
                mu_new    = y0 - sigma_new*z0
                params[i]['sigma'], params[i]['mu'] = sigma_new, mu_new
                break

    return (
        make_fig(params),
        params,
        params[0]['sigma'], params[1]['sigma'],
        params[0]['mu'],    params[1]['mu']
    )

if __name__ == '__main__':
    app.run(debug=False)
